# Final Project
### Team: Data Miner(Weirong He, Michael Gainey, Tri Pham)
### Topic: Taxi Data Analysis

In [16]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import numpy as np
import pyarrow.parquet as pq

In [2]:
sc = SparkContext()
session = SparkSession(sc)

21/11/16 12:49:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


## Data Process
Data set could be download at https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page
In this project, we are focusing on year 2020, so we have 12 month taxi data in 2020. However, this dataset is too large for this project, so we decide to only take data at the first week of each month which will lead to result of 5M records.

In [3]:
taxi_record_1 = session.read.csv("data/records/yellow_tripdata_2020-01.csv", header=True)
taxi_record_2 = session.read.csv("data/records/yellow_tripdata_2020-02.csv", header=True)
taxi_record_3 = session.read.csv("data/records/yellow_tripdata_2020-03.csv", header=True)
taxi_record_4 = session.read.csv("data/records/yellow_tripdata_2020-04.csv", header=True)
taxi_record_5 = session.read.csv("data/records/yellow_tripdata_2020-05.csv", header=True)
taxi_record_6 = session.read.csv("data/records/yellow_tripdata_2020-06.csv", header=True)
taxi_record_7 = session.read.csv("data/records/yellow_tripdata_2020-07.csv", header=True)
taxi_record_8 = session.read.csv("data/records/yellow_tripdata_2020-08.csv", header=True)
taxi_record_9 = session.read.csv("data/records/yellow_tripdata_2020-09.csv", header=True)
taxi_record_10 = session.read.csv("data/records/yellow_tripdata_2020-10.csv", header=True)
taxi_record_11 = session.read.csv("data/records/yellow_tripdata_2020-11.csv", header=True)
taxi_record_12 = session.read.csv("data/records/yellow_tripdata_2020-12.csv", header=True)
location_df = session.read.csv("data/taxi+_zone_lookup.csv", header=True)

In [4]:
dates = ("2020-01-01",  "2020-01-07")
taxi_record_1 = taxi_record_1.where(F.col('tpep_pickup_datetime').between(*dates))
dates = ("2020-02-01",  "2020-02-07")
taxi_record_2 = taxi_record_2.where(F.col('tpep_pickup_datetime').between(*dates))
dates = ("2020-03-01",  "2020-03-07")
taxi_record_3 = taxi_record_3.where(F.col('tpep_pickup_datetime').between(*dates))
dates = ("2020-04-01",  "2020-04-07")
taxi_record_4 = taxi_record_4.where(F.col('tpep_pickup_datetime').between(*dates))
dates = ("2020-05-01",  "2020-05-07")
taxi_record_5 = taxi_record_5.where(F.col('tpep_pickup_datetime').between(*dates))
dates = ("2020-06-01",  "2020-06-07")
taxi_record_6 = taxi_record_6.where(F.col('tpep_pickup_datetime').between(*dates))
dates = ("2020-07-01",  "2020-07-07")
taxi_record_7 = taxi_record_7.where(F.col('tpep_pickup_datetime').between(*dates))
dates = ("2020-08-01",  "2020-08-07")
taxi_record_8 = taxi_record_8.where(F.col('tpep_pickup_datetime').between(*dates))
dates = ("2020-09-01",  "2020-09-07")
taxi_record_9 = taxi_record_9.where(F.col('tpep_pickup_datetime').between(*dates))
dates = ("2020-10-01",  "2020-10-07")
taxi_record_10 = taxi_record_10.where(F.col('tpep_pickup_datetime').between(*dates))
dates = ("2020-11-01",  "2020-11-07")
taxi_record_11 = taxi_record_11.where(F.col('tpep_pickup_datetime').between(*dates))
dates = ("2020-12-01",  "2020-12-07")
taxi_record_12 = taxi_record_12.where(F.col('tpep_pickup_datetime').between(*dates))

In [5]:
taxi_record = (taxi_record_1.union(taxi_record_2).union(taxi_record_3)
               .union(taxi_record_4).union(taxi_record_5).union(taxi_record_6)
               .union(taxi_record_7).union(taxi_record_8).union(taxi_record_9)
               .union(taxi_record_10).union(taxi_record_11).union(taxi_record_12)
               .drop("VendorID", "RatecodeID", "payment_type", "store_and_fwd_flag",
                     "improvement_surcharge", "mta_tax", "tolls_amount", "congestion_surcharge")
              )

In [6]:
taxi_record.head()

Row(tpep_pickup_datetime='2020-01-01 00:28:15', tpep_dropoff_datetime='2020-01-01 00:33:03', passenger_count='1', trip_distance='1.20', PULocationID='238', DOLocationID='239', fare_amount='6', extra='3', tip_amount='1.47', total_amount='11.27')

### Machine Learning
We can use Machine Learning to help taxi driver to predict the fare amout of next trip. In real life, taxi passengers alway like to ask the price right after they stating the destination location. Many drivers with more experience can quickly response a rough price. And this model can look through 5M trips to become a experienced taxi driver and make a prediction of the fare amount based on the pickup location and drop off location.

In [19]:
# data process
loc_price = taxi_record.select(F.col("PULocationID").alias("PU"),
                   F.col("DOLocationID").alias("DO"),
                   F.col("fare_amount").alias("Price"))
# filter null value record and unknown record(locationID: 264 and 265 are unknown)
loc_price = loc_price.filter(loc_price.PU.isNotNull() & loc_price.PU.between(1, 263)
                             & loc_price.DO.isNotNull() & loc_price.DO.between(1, 263)
                             & loc_price.Price.isNotNull())

In [8]:
# locations = np.array(loc_price.select("PU", "DO").collect())
# price = np.array(loc_price.select("Price").collect())

In [24]:
%%time
import pyarrow as pa
import pyarrow.parquet as pq
loc_price.write.mode("overwrite").parquet("loc_price.parquet")
pd_df = pq.read_pandas('loc_price.parquet').to_pandas()

CPU times: user 1.03 s, sys: 268 ms, total: 1.29 s
Wall time: 33.2 s


In [29]:
pickup = np.array(pd_df.PU)
dropoff = np.array(pd_df.DO)
price = np.array(pd_df.Price)

In [37]:
location = np.vstack((pickup, dropoff)).T

In [35]:
x

array([['238', '239'],
       ['239', '238'],
       ['238', '238'],
       ...,
       ['48', '76'],
       ['219', '35'],
       ['45', '122']], dtype=object)

In [36]:
price

array(['6', '7', '6', ..., '32.32', '39.45', '59.87'], dtype=object)

In [40]:
%%time
from sklearn.model_selection import train_test_split
# data split
X_train, X_test, y_train, y_test = train_test_split(location, price, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.125)

CPU times: user 4.58 s, sys: 117 ms, total: 4.69 s
Wall time: 4.67 s


In [42]:
from sklearn.linear_model import SGDClassifier

In [45]:
test = SGDClassifier(loss="hinge", penalty="l2")
X = np.array([[1, 2], [1, 9], [2, 3]]
y = [6, 20, 6]
test.fit(X, y)

SGDClassifier()

In [50]:
test.predict([[3,8]])

array([6])

In [41]:
%%time
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(loss="hinge", penalty="l2", max_iter=5)
clf.fit(X_train, y_train)

KeyboardInterrupt: 